In [1]:
import pandas as pd
from transformers import BertTokenizer,BertModel
import re
from unidecode import unidecode
from nltk.corpus import stopwords

In [2]:
data=pd.read_csv("sampled_data.csv")

In [3]:
data

,Category,Content
0,Middle East,"""""Sadness, anger in Palestine: Gaza mourns 13 ..."
1,Middle East,"Sudan’s doctors battle to keep people alive,""""..."
2,Middle East,How has the world reacted to Syria rejoining t...
3,Middle East,Police question two Tunisian journalists after...
4,Middle East,United Nations backs Sudan envoy as army seeks...
...,...,...
4995,Science & Technology,Facebook employees demand change around Palest...
4996,Science & Technology,US stocks close on upbeat data ahead of tech e...
4997,Science & Technology,Could this gene double your risk of dying from...
4998,Science & Technology,"""""Zoom, Amazon, ransomware: Tech’s big winners..."


In [3]:
# Compter le nombre d'enregistrements par catégorie
category_counts = data.groupby('Category').size()

# Afficher le résultat
print(category_counts)

Category
Africa                  500
Asia                    500
Climate Crisis          500
Coronavirus             500
Economy                 500
Europe                  500
Middle East             500
Opinion                 500
Science & Technology    500
Sports                  500
dtype: int64


In [6]:
# Charger la liste des stopwords
stop_words = set(stopwords.words('english'))  # Utiliser une autre langue si nécessaire

In [7]:
# Nettoyer les données textuelles
def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer les accents
    text = unidecode(text)
    
    # Retirer les guillemets
    text = text.replace('"', '').replace("'", "")
    
    # Supprimer les balises HTML
    text = re.sub(r"<.*?>", " ", text)
    
    # Supprimer les URL
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    
    # Supprimer les caractères spéciaux (conserver les mots et espaces)
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    
    # Supprimer les chiffres
    text = re.sub(r"\d+", " ", text)
    
    # Supprimer les mots courts (moins de 2 caractères, sauf "a" ou "i")
    text = re.sub(r"\b\w{1,1}\b", " ", text)
    
    # Retirer les espaces multiples
    text = re.sub(r"\s+", " ", text).strip()
    
    # Supprimer les stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)
    
    return text

In [8]:
data['Content'] = data['Content'].apply(clean_text)

In [9]:
# Charger le tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
# Tokeniser les textes
def tokenize_function(texts, max_length=128):
    return tokenizer(
        texts.tolist(),  # Les textes sous forme de liste
        padding=True,  # Ajouter des pads pour avoir une longueur fixe
        truncation=True,  # Tronquer les textes dépassant `max_length`
        max_length=max_length,  # Longueur maximale autorisée
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

In [11]:
# Tokenisation des textes dans la colonne 'Content'
tokens = tokenize_function(data['Content'])

# Vérifier les tokens générés
print("Tokens générés :", tokens.keys())

Tokens générés : dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [12]:
# Sauvegarder les tokens pour une utilisation ultérieure
import torch
torch.save(tokens, 'bert_tokens_sans2.pt')

print("Les données sont prêtes pour BERT et sauvegardées dans 'bert_tokens_sans2.pt'.")

Les données sont prêtes pour BERT et sauvegardées dans 'bert_tokens_sans2.pt'.


In [13]:
# Charger le modèle pré-entraîné BERT
model = BertModel.from_pretrained('bert-base-uncased')

In [14]:
# Calcul des embeddings avec BERT
with torch.no_grad():
    outputs = model(**tokens)  # Passer les tokens dans BERT
    embeddings = outputs.last_hidden_state  # Les embeddings de chaque token

In [16]:
print(embeddings)

tensor([[[-1.5784e-01,  4.2919e-02,  1.8769e-01,  ..., -3.4746e-01,
           2.5236e-01,  1.5060e-02],
         [-1.7151e-01,  6.2319e-01,  4.4668e-01,  ..., -2.5251e-01,
          -6.4288e-03, -4.1159e-01],
         [ 7.7168e-03,  4.7289e-01,  3.4458e-01,  ..., -3.1900e-01,
          -2.0013e-01, -6.6254e-01],
         ...,
         [ 4.0123e-01,  1.4561e-01,  2.3749e-01,  ..., -3.8078e-01,
           1.3876e-01,  2.2495e-02],
         [-9.6551e-01, -3.6961e-01, -6.4580e-02,  ..., -1.7299e-01,
           1.1782e-01,  5.1195e-02],
         [ 6.7091e-01,  2.6258e-01, -6.7924e-02,  ..., -1.6575e-01,
          -1.7991e-01,  2.0322e-02]],

        [[-5.5316e-01, -1.6308e-02,  1.3948e-01,  ..., -5.0167e-01,
           5.5180e-01, -6.0400e-02],
         [ 4.3219e-02,  7.2143e-01, -1.6046e-01,  ..., -3.9942e-01,
          -3.9348e-02,  3.5283e-02],
         [-9.8077e-02, -1.1632e-01,  2.2951e-01,  ..., -1.5507e-01,
           1.9858e-02, -2.3513e-01],
         ...,
         [-5.7736e-01,  3

In [17]:
import numpy as np
# Convertir les embeddings en NumPy
embeddings_numpy = embeddings.numpy()

# Enregistrer dans un fichier .npy
np.save("embeddings.npy", embeddings_numpy)

# Charger les embeddings plus tard
loaded_embeddings = np.load("embeddings.npy")